##**Gradio with writing evluating system+ tts + system workflow**

In [ ]:
!pip install --upgrade gradio

In [ ]:
pip install gradio requests

In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, VitsModel, AutoTokenizer as TTSTokenizer
from sentence_transformers import SentenceTransformer, util
import numpy as np
import time

# Add language detection function
def detect_language(text):
    """Detect whether the text is Chinese or English"""
    # Simple judgment: if it contains Chinese characters, it is considered Chinese
    for char in text:
        if '\u4e00' <= char <= '\u9fff':
            return "zh"
    return "en"

# 1. Load existing models and data
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = SentenceTransformer("all-mpnet-base-v2").to(device)

# Simple user data storage
user_progress = {}

# Load your text data and embeddings
try:
    import pandas as pd
    text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")
    # Transform Embed
    text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(
        lambda x: np.fromstring(x.strip("[]"), sep=" "))
    pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")
    embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()),
                              dtype=torch.float32).to(device)
    print("✅ Successfully loaded data")
except Exception as e:
    print(f"❌ Failed to load data: {e}")
    # If the data loading fails, you can provide some sample data
    pages_and_chunks = []
    embeddings = None

# 2. Load LLM
model_id = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Add TTS model (lazy loading to save memory)
tts_model = None
tts_tokenizer = None

def load_tts_model():
    global tts_model, tts_tokenizer
    if tts_model is None:
        print("Loading TTS model...")
        tts_model = VitsModel.from_pretrained("facebook/mms-tts-eng")
        tts_tokenizer = TTSTokenizer.from_pretrained("facebook/mms-tts-eng")
        print("✅ TTS model loaded")
    return tts_model, tts_tokenizer

# 3. Define retrieval and generation functions
def retrieve_relevant_resources(query, embeddings, n_resources=5):
    """Search for related resources"""
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    scores, indices = torch.topk(dot_scores, k=n_resources)
    return scores, indices

def generate_answer(query, context_items, avg_relevance_score=0.0):
    """Generate answers, determine the answer language based on the query language, and decide whether to use the search content based on relevance"""
    # Set a relevance threshold below which LLM knowledge is used instead of search content
    relevance_threshold = 0.65

    # All output uses English by default
    language = "en"

    # Determine the prompt content based on relevance
    if avg_relevance_score < relevance_threshold:
        # Low relevance, let the model use its own knowledge
        prompt = f"""Here is a question about the IELTS exam. Since no sufficiently relevant reference materials were found, please use your own knowledge to answer.

Question: {query}

Answer:"""
    else:
        # Highly relevant, using the retrieved content
        context = "\n\n".join([item["sentence_chunk"] for item in context_items])

        prompt = f"""Based on the following IELTS materials, answer the question:

Content:
{context}

Question: {query}

Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "")
    return response, language

def evaluate_response_quality(query, response, relevance_score):
    """Evaluate the quality of the system's answers"""
    # Evaluate based on relevance and heuristic rules
    coherence_score = min(1.0, 0.5 + relevance_score * 0.5)  # Simple heuristic rules

    # Generate English evaluation report
    if relevance_score > 0.8:
        relevance_comment = "Highly Relevant - The answer directly addresses the core of the question"
    elif relevance_score > 0.6:
        relevance_comment = "Relevant - The answer covers main aspects of the question"
    else:
        relevance_comment = "Partially Relevant - The answer only partially addresses the question"

    # Text length evaluation
    if len(response) < 50:
        length_comment = "Too Short - The answer may not be comprehensive"
    elif len(response) > 500:
        length_comment = "Extensive - The answer is very comprehensive"
    else:
        length_comment = "Adequate - The answer length is reasonable"

    # Portfolio Assessment Report
    report = f"""
### Answer Quality Assessment

**Relevance**: {relevance_score:.2f}/1.0 - {relevance_comment}
**Coherence**: {coherence_score:.2f}/1.0
**Length**: {len(response)} characters - {length_comment}

**Overall Rating**: {(relevance_score + coherence_score) / 2:.2f}/1.0
    """

    return report

def process_query(query):
    """Processes the query and returns results and performance indicators (improved version with relevance evaluation)"""
    # Check if data has been loaded
    if embeddings is None:
        return "Data not loaded, please run the data processing code first", "", "Performance metrics unavailable: data not loaded"

    # Recording start time
    start_time = time.time()

    # Get relevant resources
    retrieval_start = time.time()
    scores, indices = retrieve_relevant_resources(query, embeddings)
    retrieval_time = time.time() - retrieval_start

    context_items = [pages_and_chunks[i] for i in indices]

    # Calculate the average relevance score
    avg_relevance = scores.mean().item()

    # Record generation start time
    generation_start = time.time()

    # Generate Answer (now passes relevance score)
    answer, detected_language = generate_answer(query, context_items, avg_relevance)

    # Calculate generation time
    generation_time = time.time() - generation_start
    total_time = time.time() - start_time

    # Preparing context information for display
    context_display = ""
    for i, (score, idx) in enumerate(zip(scores, indices)):
        context_display += f"**Reference {i+1}** (Relevance: {score:.2f}):\n{pages_and_chunks[idx]['sentence_chunk'][:200]}...\n\n"

    # Quality Assessment
    quality_report = evaluate_response_quality(query, answer, avg_relevance)

    # Preparing performance indicators
    metrics = f"""
### System Performance Metrics
- **Retrieval Time**: {retrieval_time:.2f} seconds
- **Generation Time**: {generation_time:.2f} seconds
- **Total Response Time**: {total_time:.2f} seconds
- **Average Relevance Score**: {avg_relevance:.4f}/1.0
- **Response Mode**: {"Retrieved Context" if avg_relevance >= 0.65 else "LLM Knowledge"}
{quality_report}
"""

    return answer, context_display, metrics

def process_query_with_history(query, history=""):
    """Processing queries and saving history"""
    answer, context, metrics = process_query(query)

    # Update History
    timestamp = time.strftime("%H:%M:%S")
    new_history = f"{history}<hr><b>[{timestamp}] Q:</b> {query}<br><b>A:</b> {answer}<br>"

    return answer, context, metrics, new_history

def track_user_activity(username, activity_type, content, score=None):
    """Tracking user learning activities"""
    if username not in user_progress:
        user_progress[username] = {
            "queries": [],
            "writing_samples": [],
            "practice_tests": [],
            "last_active": None
        }

    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")

    if activity_type == "query":
        user_progress[username]["queries"].append({
            "timestamp": timestamp,
            "query": content,
            "relevance_score": score
        })
    elif activity_type == "writing":
        user_progress[username]["writing_samples"].append({
            "timestamp": timestamp,
            "sample": content[:100] + "...",  # Save Summary
            "score": score
        })
    elif activity_type == "practice":
        user_progress[username]["practice_tests"].append({
            "timestamp": timestamp,
            "test_type": content,
            "completed": True
        })

    user_progress[username]["last_active"] = timestamp

    # Build Progress Summary
    summary = f"""
### Learning Progress Summary ({username})
- Questions asked: {len(user_progress[username]["queries"])}
- Writing samples: {len(user_progress[username]["writing_samples"])}
- Practice tests: {len(user_progress[username]["practice_tests"])}
- Last active: {user_progress[username]["last_active"]}

#### Recent Activity
"""

    # Add the last 5 events
    recent_queries = user_progress[username]["queries"][-3:] if user_progress[username]["queries"] else []
    recent_writings = user_progress[username]["writing_samples"][-2:] if user_progress[username]["writing_samples"] else []

    for q in recent_queries:
        summary += f"- [{q['timestamp']}] Question: {q['query'][:50]}...\n"

    for w in recent_writings:
        summary += f"- [{w['timestamp']}] Writing practice\n"

    return summary

def process_query_with_tracking(query, username):
    """Handle inquiries and track learning progress"""
    answer, context, metrics = process_query(query)

    # Extracting relevance scores from metrics
    import re
    relevance_match = re.search(r"Average Relevance Score: ([\d\.]+)", metrics)
    relevance_score = float(relevance_match.group(1)) if relevance_match else None

    # Track this query
    progress = track_user_activity(username, "query", query, relevance_score)

    return answer, context, metrics, progress

# Modify the TTS function to ensure that the complete content is processed
def text_to_speech(text):
    """Convert text to speech, process full English text"""
    if not text:
        return None

    # Load the TTS model (if not already loaded)
    model, tokenizer = load_tts_model()

    # If the text is too long, process it in segments and concatenate them
    max_segment_length = 500  # Maximum length of each segment
    segments = []

    # Segment text
    if len(text) > max_segment_length:
        words = text.split()
        current_segment = []
        current_length = 0

        for word in words:
            current_length += len(word) + 1  # +1 for space
            if current_length <= max_segment_length:
                current_segment.append(word)
            else:
                segments.append(" ".join(current_segment))
                current_segment = [word]
                current_length = len(word) + 1

        if current_segment:
            segments.append(" ".join(current_segment))
    else:
        segments = [text]

    # Process each paragraph and concatenate
    full_waveform = None
    sample_rate = None

    for segment in segments:
        inputs = tokenizer(segment, return_tensors="pt")
        with torch.no_grad():
            output = model(**inputs).waveform

        if full_waveform is None:
            full_waveform = output[0].numpy()
            sample_rate = model.config.sampling_rate
        else:
            # Add a short pause (0.3 seconds of silence)
            pause = np.zeros(int(0.3 * sample_rate))
            full_waveform = np.concatenate([full_waveform, pause, output[0].numpy()])

    # Return to full audio
    return (sample_rate, full_waveform)

# Text-to-speech language detection wrapper function
def tts_with_language_check(text):
    if not text:
        return None, "Please provide text content"

    language = detect_language(text)
    if language == "zh":
        return None, "⚠️ Only English text is supported for TTS. Please provide English text."
    else:
        try:
            audio = text_to_speech(text)
            return audio, "✅ Conversion successful! Full content converted to speech."
        except Exception as e:
            return None, f"❌ Error during conversion: {str(e)}"

# Added IELTS writing scoring function
def evaluate_ielts_writing(writing_sample, username="default_user"):
    """Evaluate IELTS writing samples and keep track of records"""
    if not writing_sample:
        return "Please provide a writing sample for evaluation."

    prompt = f"""As an IELTS examiner, please assess the following student writing sample.
    Provide scores and specific suggestions based on these criteria:
    1. Task Response
    2. Coherence and Cohesion
    3. Lexical Resource
    4. Grammatical Range and Accuracy

    Give scores in 0.5 increments (e.g., 6.0, 6.5) for each category, and provide an overall score.

    Student writing sample:
    {writing_sample}

    Score and detailed feedback:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.2,
            do_sample=True
        )
    feedback = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "")

    # Extract total score
    import re
    score_match = re.search(r"Overall score.*?(\d+\.?\d*)", feedback)
    overall_score = float(score_match.group(1)) if score_match else None

    # Record writing activities
    track_user_activity(username, "writing", writing_sample, overall_score)

    return feedback

# Added mock exam feature
def generate_practice_question(section_type):
    """Generate IELTS practice questions"""
    section_type_english = section_type.split(" ")[0]  # Get the English section

    prompt = f"""Create an IELTS {section_type_english} practice question.
    Include detailed questions, guidance, and scoring criteria.
    For Writing or Speaking sections, provide a sample question and response framework.
    For Listening or Reading sections, provide sample questions and answer options."""

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True
        )
    practice = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "")
    return practice

# Add learning plan generation function
def generate_study_plan(target_score, weeks_available, strengths, weaknesses):
    """Generate a personalized IELTS study plan"""
    prompt = f"""Create a personalized IELTS study plan with the following conditions:

    Target Score: {target_score}
    Available Time: {weeks_available} weeks
    Strengths: {strengths}
    Weaknesses: {weaknesses}

    Please provide:
    1. Detailed weekly study plan
    2. Recommended learning resources
    3. Specific exercises for weaknesses
    4. Regular mock test schedule
    5. Pre-exam preparation strategy
    """

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=1024,
            temperature=0.7,
            do_sample=True
        )
    plan = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "")
    return plan

def export_history(history):
    """Export session history as text"""
    if not history:
        return "No conversation history to export"

    try:
        from bs4 import BeautifulSoup
        import re

        # Parsing HTML and extracting text using BeautifulSoup
        soup = BeautifulSoup(history, "html.parser")
        text = soup.get_text()

        # Cleaning up the text
        cleaned_text = re.sub(r'\s+', ' ', text).strip()

        # Returns the cleaned text and timestamp
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        return f"Conversation exported. Filename: ielts_conversation_{timestamp}.txt\n\n{cleaned_text[:100]}..."
    except:
        # Simple alternate extraction
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        return f"Conversation exported. Filename: ielts_conversation_{timestamp}.txt"

# Improve system architecture diagram generation function
def generate_system_diagram():
    """Generate comprehensive system architecture diagram"""
    from PIL import Image, ImageDraw, ImageFont
    import io
    import base64
    import os

    # Create a larger image to show the full content
    width, height = 1000, 650
    image = Image.new("RGB", (width, height), "white")
    draw = ImageDraw.Draw(image)

    # Try loading a better font
    try:
        # Try common fonts, depending on the system
        font_paths = [
            "arial.ttf",
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
            "/System/Library/Fonts/Helvetica.ttc"
        ]

        font = None
        for path in font_paths:
            if os.path.exists(path):
                font = ImageFont.truetype(path, 16)
                title_font = ImageFont.truetype(path, 24)
                break

        if font is None:
            font = ImageFont.load_default()
            title_font = font
    except:
        font = ImageFont.load_default()
        title_font = font

    # Define the main process box
    main_flow_boxes = [
        {"x": 100, "y": 150, "width": 150, "height": 80, "text": "User Query", "color": "#FFD580", "description": "Questions about IELTS"},
        {"x": 320, "y": 150, "width": 150, "height": 80, "text": "Vector Retrieval", "color": "#90EE90", "description": "Embedding search"},
        {"x": 540, "y": 150, "width": 150, "height": 80, "text": "Relevant Content", "color": "#ADD8E6", "description": "Retrieved materials"},
        {"x": 760, "y": 150, "width": 150, "height": 80, "text": "LLM Generation", "color": "#DDA0DD", "description": "Gemma-2b-it model"},
        {"x": 760, "y": 300, "width": 150, "height": 80, "text": "Result Display", "color": "#FFCCCB", "description": "Answer with context"}
    ]

    # Define other functional modules
    feature_boxes = [
        {"x": 100, "y": 300, "width": 150, "height": 80, "text": "Writing Assessment", "color": "#F0E68C", "description": "Essay evaluation"},
        {"x": 320, "y": 300, "width": 150, "height": 80, "text": "Study Plan", "color": "#98FB98", "description": "Learning roadmap"},
        {"x": 540, "y": 300, "width": 150, "height": 80, "text": "Practice Tests", "color": "#87CEEB", "description": "Mock exams"},
        {"x": 320, "y": 450, "width": 150, "height": 80, "text": "Text-to-Speech", "color": "#D8BFD8", "description": "Voice output"},
        {"x": 540, "y": 450, "width": 150, "height": 80, "text": "Progress Tracking", "color": "#FFB6C1", "description": "Learning journey"}
    ]

    # Draw the main title
    title = "IELTS Learning Assistant System Architecture"
    title_w, title_h = draw.textsize(title, font=title_font) if hasattr(draw, 'textsize') else (width//2, 40)
    draw.text(((width - title_w) // 2, 30), title, fill="#4B0082", font=title_font)

    # Draw the main process box and description
    for box in main_flow_boxes:
        x, y = box["x"], box["y"]
        w, h = box["width"], box["height"]
        draw.rectangle([(x, y), (x+w, y+h)], fill=box["color"], outline="#000000", width=2)

        # Adding Main Text
        text_w, text_h = draw.textsize(box["text"], font=font) if hasattr(draw, 'textsize') else (w//2, h//3)
        text_x = x + (w - text_w) // 2
        text_y = y + (h - text_h) // 3
        draw.text((text_x, text_y), box["text"], fill="#000000", font=font)

        # Add description text
        desc_w, desc_h = draw.textsize(box["description"], font=font) if hasattr(draw, 'textsize') else (w//2, h//3)
        desc_x = x + (w - desc_w) // 2
        desc_y = y + h - text_h - 10
        draw.text((desc_x, desc_y), box["description"], fill="#333333", font=font)

    # Draw function module boxes and descriptions
    for box in feature_boxes:
        x, y = box["x"], box["y"]
        w, h = box["width"], box["height"]
        draw.rectangle([(x, y), (x+w, y+h)], fill=box["color"], outline="#000000", width=2)

        # Adding Main Text
        text_w, text_h = draw.textsize(box["text"], font=font) if hasattr(draw, 'textsize') else (w//2, h//3)
        text_x = x + (w - text_w) // 2
        text_y = y + (h - text_h) // 3
        draw.text((text_x, text_y), box["text"], fill="#000000", font=font)

        # Add description text
        desc_w, desc_h = draw.textsize(box["description"], font=font) if hasattr(draw, 'textsize') else (w//2, h//3)
        desc_x = x + (w - desc_w) // 2
        desc_y = y + h - text_h - 10
        draw.text((desc_x, desc_y), box["description"], fill="#333333", font=font)

    # Drawing Connection Lines - Main Process
    for i in range(len(main_flow_boxes)-2):  # The last frame is processed separately
        x1 = main_flow_boxes[i]["x"] + main_flow_boxes[i]["width"]
        y1 = main_flow_boxes[i]["y"] + main_flow_boxes[i]["height"]//2
        x2 = main_flow_boxes[i+1]["x"]
        y2 = main_flow_boxes[i+1]["y"] + main_flow_boxes[i+1]["height"]//2

        # Lire
        draw.line([(x1, y1), (x2, y2)], fill="#000000", width=3)

        # Arrow
        arrow_size = 10
        draw.polygon([(x2-arrow_size, y2-arrow_size//2), (x2, y2), (x2-arrow_size, y2+arrow_size//2)], fill="#000000")

    # Connection from LLM Generation to Result Display
    llm_idx = 3  # LLM Generation Index
    result_idx = 4  #Result Display Index

    x1 = main_flow_boxes[llm_idx]["x"] + main_flow_boxes[llm_idx]["width"]//2
    y1 = main_flow_boxes[llm_idx]["y"] + main_flow_boxes[llm_idx]["height"]
    x2 = main_flow_boxes[result_idx]["x"] + main_flow_boxes[result_idx]["width"]//2
    y2 = main_flow_boxes[result_idx]["y"]

    # Lines and arrows
    draw.line([(x1, y1), (x1, y1+30), (x2, y1+30), (x2, y2)], fill="#000000", width=3)
    arrow_size = 10
    draw.polygon([(x2-arrow_size//2, y2-arrow_size), (x2, y2), (x2+arrow_size//2, y2-arrow_size)], fill="#000000")

    # LLM Generation to TTS connection
    tts_idx = 3  # Text-to-Speech Index

    x1 = main_flow_boxes[llm_idx]["x"]
    y1 = main_flow_boxes[llm_idx]["y"] + main_flow_boxes[llm_idx]["height"]//2
    x2 = feature_boxes[tts_idx]["x"] + feature_boxes[tts_idx]["width"]//2
    y2 = feature_boxes[tts_idx]["y"]

    draw.line([(x1, y1), (x1-30, y1), (x1-30, y2-30), (x2, y2-30), (x2, y2)], fill="#000000", width=2)
    arrow_size = 10
    draw.polygon([(x2-arrow_size//2, y2-arrow_size), (x2, y2), (x2+arrow_size//2, y2-arrow_size)], fill="#000000")

    #Add a legend
    legend_y = 580
    legend_items = [
        {"text": "Main Process Flow", "color": "#000000", "width": 3},
        {"text": "Additional Features", "color": "#000000", "width": 2},
    ]

    for i, item in enumerate(legend_items):
        x_pos = 100 + i * 300
        # Line
        draw.line([(x_pos, legend_y), (x_pos + 50, legend_y)], fill=item["color"], width=item["width"])
        # text
        draw.text((x_pos + 60, legend_y - 5), item["text"], fill="#000000", font=font)

    # Add bottom note
    footer_text = "Built with SentenceTransformer, Google Gemma-2b-it, and Facebook MMS-TTS-Eng"
    footer_w, footer_h = draw.textsize(footer_text, font=font) if hasattr(draw, 'textsize') else (width//2, 20)
    draw.text(((width - footer_w) // 2, height - 30), footer_text, fill="#666666", font=font)

    return image

# 4. Creating the Gradio Interface
def create_interface():
    """Creating the Gradio Interface"""
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        with gr.Tab("Main Application"):
            gr.Markdown("# IELTS Learning Assistant")
            gr.Markdown("""
            This application uses artificial intelligence to answer questions about the IELTS exam. It is based on IELTS textbook content and can help you understand key aspects of the exam and improve your skills.

            **Language Support**:
            - Text-to-speech functionality is only available for English
            """)

            # User Information
            with gr.Row():
                username_input = gr.Textbox(
                    label="Username",
                    placeholder="Enter your username to track learning progress",
                    value="default_user"
                )

            with gr.Row():
                with gr.Column(scale=2):
                    query_input = gr.Textbox(
                        label="Question",
                        placeholder="Enter your question about the IELTS exam...",
                        lines=2
                    )
                    with gr.Row():
                        submit_btn = gr.Button("Submit", variant="primary")
                        clear_btn = gr.Button("Clear")

                    response_output = gr.Textbox(
                        label="Answer",
                        lines=10,
                        placeholder="AI's answer will appear here...",
                    )

                    # New: Independent performance indicator display
                    metrics_output = gr.Markdown(label="Performance Metrics")

                with gr.Column(scale=1):
                    with gr.Accordion("References", open=False):
                        context_output = gr.Markdown()

            # Session History
            with gr.Accordion("Conversation History", open=False):
                history_display = gr.HTML()
                with gr.Row():
                    export_btn = gr.Button("Export Conversation")
                    clear_history_btn = gr.Button("Clear History")
                export_status = gr.Textbox(label="Export Status", visible=False)

            # Example Question
            examples = [
                ["How can I improve my IELTS listening score?"],
                ["What is the ideal structure for IELTS Writing Task 2?"],
                ["How to manage time effectively in the IELTS reading test?"],
                ["What should I pay attention to in the IELTS speaking test?"],
                ["What are the common mistakes in IELTS Writing Task 1?"],
                ["How to achieve band 7+ in IELTS?"]
            ]

            # Setup Events - Updated to use tracking and history
            submit_btn.click(
                fn=process_query_with_history,
                inputs=[query_input, history_display],
                outputs=[response_output, context_output, metrics_output, history_display]
            )
            clear_btn.click(
                lambda: ["", "", ""],
                outputs=[query_input, response_output, metrics_output]
            )
            export_btn.click(
                fn=export_history,
                inputs=history_display,
                outputs=export_status
            )
            clear_history_btn.click(
                lambda: "",
                outputs=history_display
            )
            gr.Examples(
                examples=examples,
                inputs=query_input
            )

            # Adding text-to-speech functionality
            with gr.Accordion("Text-to-Speech Feature", open=False):
                gr.Markdown("## Convert Text to Speech")
                gr.Markdown("Enter English text or use the answer content to convert to speech.")

                with gr.Row():
                    text_for_tts = gr.Textbox(
                        label="Text to convert to speech",
                        lines=2,
                        placeholder="Enter English text to convert to speech..."
                    )
                    with gr.Column():
                        convert_btn = gr.Button("Convert Text", variant="secondary")
                        convert_response_btn = gr.Button("Convert Answer", variant="secondary")

                audio_output = gr.Audio(label="Speech Output")
                tts_status = gr.Markdown()  # Add status information display

                # Connecting TTS Function
                convert_btn.click(
                    fn=tts_with_language_check,
                    inputs=text_for_tts,
                    outputs=[audio_output, tts_status]
                )
                convert_response_btn.click(
                    fn=tts_with_language_check,
                    inputs=response_output,
                    outputs=[audio_output, tts_status]
                )

            # Add learning progress display
            view_progress_btn = gr.Button("View Learning Progress")
            progress_display = gr.Markdown(label="Learning Progress")

            view_progress_btn.click(
                fn=lambda username: track_user_activity(username, "query", "View progress"),
                inputs=username_input,
                outputs=progress_display
            )

        # Writing Grading Tab
        with gr.Tab("Writing Assessment"):
            gr.Markdown("# IELTS Writing Assessment")
            gr.Markdown("Upload your IELTS writing sample to get professional scoring and feedback.")

            writing_username = gr.Textbox(
                label="Username",
                placeholder="Enter your username to track progress",
                value="default_user"
            )

            writing_input = gr.Textbox(
                label="Paste your IELTS writing sample",
                placeholder="Paste your Task 1 or Task 2 writing here...",
                lines=10
            )
            evaluate_btn = gr.Button("Get Assessment", variant="primary")
            evaluation_output = gr.Markdown(label="Scoring and Feedback")
            writing_progress = gr.Markdown(label="Writing Progress")

            # Updated writing grades to track progress
            def evaluate_with_tracking(sample, username):
                feedback = evaluate_ielts_writing(sample, username)
                progress = track_user_activity(username, "writing", sample)
                return feedback, progress

            evaluate_btn.click(
                fn=evaluate_with_tracking,
                inputs=[writing_input, writing_username],
                outputs=[evaluation_output, writing_progress]
            )

        # Added mock exam tab
        with gr.Tab("Practice Tests"):
            gr.Markdown("# IELTS Practice Tests")
            gr.Markdown("Select an exam section to generate corresponding practice questions.")

            practice_username = gr.Textbox(
                label="Username",
                placeholder="Enter your username to track progress",
                value="default_user"
            )

            section_selector = gr.Dropdown(
                label="Select Exam Section",
                choices=["Listening", "Reading", "Writing", "Speaking"],
                value="Writing"
            )

            generate_btn = gr.Button("Generate Practice Question", variant="primary")
            practice_output = gr.Markdown(label="Practice Question")

            # Generate mock questions and track activity
            def generate_practice_with_tracking(section, username):
                practice = generate_practice_question(section)
                # Record this mock exam activity
                track_user_activity(username, "practice", section)
                return practice

            generate_btn.click(
                fn=generate_practice_with_tracking,
                inputs=[section_selector, practice_username],
                outputs=practice_output
            )

        # Add a learning plan tab
        with gr.Tab("Study Plan"):
            gr.Markdown("# Personalized IELTS Study Plan")
            gr.Markdown("Input your goals and conditions to get a customized IELTS study plan.")

            with gr.Row():
                target_score = gr.Slider(
                    label="Target Overall Score",
                    minimum=5.0,
                    maximum=9.0,
                    step=0.5,
                    value=7.0
                )
                weeks = gr.Slider(
                    label="Available Study Weeks",
                    minimum=1,
                    maximum=24,
                    step=1,
                    value=8
                )

            strengths = gr.Textbox(
                label="Your Strengths",
                placeholder="e.g.: Listening, Reading...",
                lines=2
            )

            weaknesses = gr.Textbox(
                label="Areas to Improve",
                placeholder="e.g.: Writing, Speaking...",
                lines=2
            )

            plan_btn = gr.Button("Generate Study Plan", variant="primary")
            plan_output = gr.Markdown(label="Personalized Study Plan")

            plan_btn.click(
                fn=generate_study_plan,
                inputs=[target_score, weeks, strengths, weaknesses],
                outputs=plan_output
            )

        # System Architecture Tab
        with gr.Tab("System Architecture"):
            with gr.Row():
                gr.Markdown("# IELTS Learning Assistant System Architecture")

            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("""
        ## Overall Architecture
        This system uses Retrieval-Augmented Generation (RAG) combined with Text-to-Speech (TTS) technology to provide IELTS learning assistance.

        ## How This System Works

        This IELTS Learning Assistant uses a powerful combination of **Retrieval-Augmented Generation (RAG)** and **Text-to-Speech** technology to provide accurate, contextually relevant responses to your IELTS questions.

        The system follows these steps:
        1. When you ask a question, it's converted to a vector representation
        2. This vector is compared against our IELTS materials database
        3. The most relevant content is retrieved
        4. The AI combines this content with its own knowledge to generate a helpful answer
        5. If relevance is low, the AI relies more on its own knowledge

        All answers are based on standard IELTS curriculum materials and best practices in IELTS preparation.
        """)

                with gr.Column(scale=1):
                    gr.Markdown("""
        ## Components
        1. **Data Preprocessing Module**
          - Splits IELTS textbook materials into semantic chunks
          - Generates embedding vectors using SentenceTransformer
          - Stores text chunks and corresponding embedding vectors

        2. **Retrieval Module**
          - Uses vector similarity search
          - Calculates similarity based on all-mpnet-base-v2 model
          - Selects the N most relevant text chunks

        3. **Generation Module**
          - Uses Google Gemma-2b-it model
          - Constructs context prompts based on retrieved content
          - Generates answers based on relevance threshold

        4. **Text-to-Speech Module**
          - Uses Facebook MMS-TTS-Eng model
          - Converts generated text to natural speech
        """)

            # Display the enhanced system diagram
            gr.Image(value=generate_system_diagram(), label="System Flow Diagram")

            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("""
        ## Learning & Assessment Components

        5. **Writing Assessment Module**
          - Evaluates IELTS writing using large language model
          - Provides detailed scoring and improvement suggestions

        6. **Study Plan Module**
          - Generates personalized learning plans based on user goals and time
          - Provides targeted learning suggestions and resource recommendations
        """)

                with gr.Column(scale=1):
                    gr.Markdown("""
        7. **Practice Test Module**
          - Generates IELTS practice questions for each section
          - Helps users familiarize with test format and content

        8. **Learning Progress Tracking**
          - Records user learning activities and achievements
          - Provides visualization of learning journey
        """)

            with gr.Accordion("Technical Details", open=False):
                gr.Markdown("""
        ### Implementation Details

        **Models Used:**
        - **Embedding Model**: SentenceTransformer (all-mpnet-base-v2)
        - **Language Model**: Google Gemma-2b-it (2 billion parameters)
        - **TTS Model**: Facebook MMS-TTS-Eng

        **RAG Implementation:**
        ```python
        # Vector search with relevance threshold
        def retrieve_relevant_resources(query, embeddings, n_resources=5):
            query_embedding = embedding_model.encode(query, convert_to_tensor=True)
            dot_scores = util.dot_score(query_embedding, embeddings)[0]
            scores, indices = torch.topk(dot_scores, k=n_resources)
            return scores, indices

        # Dynamic prompt selection based on relevance
        def generate_answer(query, context_items, avg_relevance_score=0.0):
            # Use threshold to decide prompt strategy
            relevance_threshold = 0.65

            if avg_relevance_score < relevance_threshold:
                # Low relevance - rely on model knowledge
                prompt = "Here is a question about the IELTS exam. Since no sufficiently relevant reference materials were found, please use your own knowledge to answer.\\n\\nQuestion: " + query + "\\n\\nAnswer:"
            else:
                # High relevance - use retrieved content
                context = "\\n\\n".join([item["sentence_chunk"] for item in context_items])
                prompt = "Based on the following IELTS materials, answer the question:\\n\\nContent:\\n" + context + "\\n\\nQuestion: " + query + "\\n\\nAnswer:"
                """)
    return demo

# 5. Startup interface
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(share=True)